![title.png](img/title.png)

![line.png](img/line.png)

![objectives.png](img/objectives.png)

![line.png](img/line.png)

![history.png](img/history.png)

![line.png](img/line.png)

![keyword_search_1](img/keyword_search_1.png)

![line.png](img/line.png)

![keyword_search_2.png](img/keyword_search_2.png)

![line.png](img/line.png)

![vector_embeddings.png](img/vector_embeddings.png)

![line.png](img/line.png)

![semantic_search.png](img/semantic_search.png)

![line.png](img/line.png)

![workshop_beginning.png](img/workshop_beginning.png)

# Setup environment

Install required libraries:

In [41]:
! pip install \
    pandas==2.2.2 \
    pyarrow==17.0.0 \
    vantage-sdk==0.9.3 \
    datasets==3.0.1


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


Setup utility functions (just execute following cell):

In [8]:
import pandas as pd
from IPython.display import display, HTML
from typing import List
from vantage_sdk.core.http.models.shopping_assistant_group_result import ShoppingAssistantGroupResult


def display_row(df: pd.DataFrame, row_index: int) -> None:
    row = df.iloc[row_index]
    html_content = """
    <style>
        .data-table {
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
            font-family: Arial, sans-serif;
        }
        .data-table td {
            padding: 8px 15px;
            border: 1px solid #ddd;
            vertical-align: top;
        }
        .data-table .field-name {
            font-weight: bold;
            width: 25%;
        }
    </style>
    <table class='data-table'>
    """

    for field, value in row.items():
        html_content += f"""
        <tr>
            <td class='field-name'>{field}</td>
            <td>{value}</td>
        </tr>
        """

    html_content += "</table>"
    display(HTML(html_content))


def display_query_results(df: pd.DataFrame, results: List[str]) -> None:
    ids = [result.id for result in results]
    df_filtered = df[df['id'].isin(ids)].copy()
    df_filtered['id'] = pd.Categorical(df_filtered['id'], categories=ids, ordered=True)
    df_filtered = df_filtered.sort_values('id')

    html_content = """
    <style>
        .product-grid {
            display: flex;
            flex-wrap: wrap;
            justify-content: flex-start;
            gap: 15px;
        }
        .product-container {
            border: 1px solid #ddd;
            padding: 10px;
            border-radius: 8px;
            background-color: #f9f9f9;
            box-shadow: 2px 2px 8px rgba(0, 0, 0, 0.1);
            width: 22%;
            box-sizing: border-box;
            overflow-y: auto;
            max-height: 400px;
        }
        .product-image {
            max-height: 250px;
            border-radius: 6px;
            margin-bottom: 10px;
            width: 100%;
            object-fit: contain;
        }
        .product-details {
            font-family: Arial, sans-serif;
            font-size: 12px;
            line-height: 1.4;
            color: #333;
            text-align: left;
        }
        .product-details strong {
            font-size: 14px;
            color: #0073e6;
        }
        @media (max-width: 768px) {
            .product-container {
                width: 48%;
            }
        }
        @media (max-width: 480px) {
            .product-container {
                width: 100%;
            }
        }
    </style>
    <h2>Search Results</h2>
    <div class='product-grid'>
    """
    for _, row in df_filtered.iterrows():
        html_content += f"""
        <div class='product-container'>
            <img src="{row['image_url']}" class='product-image'>
            <div class='product-details'>
                <strong>ID:</strong> {row['id']}<br>
                <strong>Title:</strong> {row['title']}<br>
                <strong>Color:</strong> {row['color']}<br>
                <strong>Brand:</strong> {row['brand']}<br>
                <strong>Gender:</strong> {row['gender']}<br>
                <strong>Product Type:</strong> {row['product_type']}<br>
                <strong>Style:</strong> {row['style']}<br>
                <strong>Price:</strong> {row['price']}<br>
                <strong>Price Range:</strong> {row['price_range']}<br>
                <strong>Availability:</strong> {row['availability']}<br>
                <strong>Description:</strong> {row['description']}<br>
            </div>
        </div>
        """

    html_content += "</div>"
    display(HTML(html_content))


def display_shopping_assistant_results(df: pd.DataFrame, groups: List[ShoppingAssistantGroupResult]) -> None:
    html_content = """
    <style>
        .group-title {
            font-family: Arial, sans-serif;
            font-size: 20px;
            margin: 20px 0 10px;
            color: #0073e6;
        }
        .product-group {
            overflow-x: auto;
            white-space: nowrap;
            padding-bottom: 10px;
            margin-bottom: 20px;
        }
        .product-container {
            display: inline-block;
            border: 1px solid #ddd;
            padding: 10px;
            border-radius: 8px;
            background-color: #f9f9f9;
            box-shadow: 2px 2px 8px rgba(0, 0, 0, 0.1);
            width: 200px;
            max-height: 400px;
            margin-right: 15px;
            box-sizing: border-box;
            overflow-y: auto;
            overflow-x: hidden;
            word-wrap: break-word;
        }
        .product-image {
            max-height: 150px;
            border-radius: 6px;
            margin-bottom: 10px;
            width: 100%;
            object-fit: contain;
        }
        .product-details {
            font-family: Arial, sans-serif;
            font-size: 12px;
            line-height: 1.4;
            color: #333;
            text-align: left;
            white-space: normal;
        }
        .product-details strong {
            font-size: 14px;
            color: #0073e6;
        }
        @media (max-width: 768px) {
            .product-container {
                width: 48%;
            }
        }
        @media (max-width: 480px) {
            .product-container {
                width: 100%;
            }
        }
    </style>
    """

    for group in groups:
        html_content += f"<h2 class='group-title'>{group.group_name}</h2>"
        html_content += "<div class='product-group'>"

        ids = [result.id for result in group.results]
        df_filtered = df[df['id'].isin(ids)].copy()
        df_filtered['id'] = pd.Categorical(df_filtered['id'], categories=ids, ordered=True)
        df_filtered = df_filtered.sort_values('id')

        for _, row in df_filtered.iterrows():
            html_content += f"""
            <div class='product-container'>
                <img src="{row['image_url']}" class='product-image'>
                <div class='product-details'>
                    <strong>ID:</strong> {row['id']}<br>
                    <strong>Title:</strong> {row['title']}<br>
                    <strong>Color:</strong> {row['color']}<br>
                    <strong>Brand:</strong> {row['brand']}<br>
                    <strong>Gender:</strong> {row['gender']}<br>
                    <strong>Product Type:</strong> {row['product_type']}<br>
                    <strong>Style:</strong> {row['style']}<br>
                    <strong>Price:</strong> {row['price']}<br>
                    <strong>Price Range:</strong> {row['price_range']}<br>
                    <strong>Availability:</strong> {row['availability']}<br>
                    <strong>Description:</strong> {row['description']}<br>
                </div>
            </div>
            """
        html_content += "</div>"
    display(HTML(html_content))


def display_images(image_urls: List[str]) -> None:
    html_content = """
    <style>
        .product-grid {
            display: flex;
            flex-wrap: wrap;
            justify-content: flex-start;
            gap: 15px;
        }
        .product-container {
            border: 1px solid #ddd;
            padding: 10px;
            border-radius: 8px;
            background-color: #f9f9f9;
            box-shadow: 2px 2px 8px rgba(0, 0, 0, 0.1);
            width: 22%;
            box-sizing: border-box;
            overflow-y: auto; /* Enable scrolling */
            max-height: 400px; /* Set maximum height for the product container */
        }
        .product-image {
            max-height: 250px; /* Larger image */
            border-radius: 6px;
            margin-bottom: 10px;
            width: 100%;
            object-fit: contain;
        }
        @media (max-width: 768px) {
            .product-container {
                width: 48%;
            }
        }
        @media (max-width: 480px) {
            .product-container {
                width: 100%;
            }
        }
    </style>
    <h2>Image Gallery</h2>
    <div class='product-grid'>
    """
    for image_url in image_urls:
        html_content += f"""
        <div class='product-container'>
            <img src="{image_url}" class='product-image'>
        </div>
        """
    html_content += "</div>"
    display(HTML(html_content))


# Dataset

The dataset consists of ~37k fashion items scraped from: `https://asos.com`.


In [45]:
RAW_DATASET = "smartcat/asos-data-embedded"

In [47]:
from datasets import load_dataset

data = load_dataset(RAW_DATASET, split="train")
dataset = data.to_pandas()
dataset.head(3)

,id,availability,brand,gender,product_type,style,color,price,price_range,description,image_url,title,embeddings
0,203506811,InStock,French Connection,Men,Gifts,Seasonal & Niche,BURGUNDY,6.0,Less than 20,Elevate any look with this versatile burgundy ...,https://images.asos-media.com/products/french-...,French Connection polka dot bow tie in burgundy,"[-0.03594585508108139, 0.007657572161406279, 0..."
1,205514883,InStock,Vans,Women,Tops,Casual Wear,CHOCOLATE,18.0,Less than 20,The Vans graphic tee brings vintage automotive...,https://images.asos-media.com/products/vans-ho...,Vans hotbox cars t-shirt in brown,"[-0.02801656164228916, -0.0274641215801239, 0...."
2,204520772,LimitedStock,NaaNaa,Women,Dresses,Dresses & Skirts,BLUE,8.5,Less than 20,Indulge in sultry sophistication with this sat...,https://images.asos-media.com/products/naanaa-...,NaaNaa satin halter neck mini dress in blue,"[-0.019270719960331917, -0.013030934147536755,..."


In [48]:
display_row(dataset, 1)

id,205514883
availability,InStock
brand,Vans
gender,Women
product_type,Tops
style,Casual Wear
color,CHOCOLATE
price,18.0
price_range,Less than 20
description,"The Vans graphic tee brings vintage automotive style to your casual wardrobe. This unisex t-shirt features a nostalgic hot rod car print across a soft, stretchy cotton jersey knit fabric in a rich chocolate brown hue. Cut with a regular, relaxed fit and classic crew neckline, it offers breathable comfort and skater-inspired aesthetic. The iconic Vans logo branding lends a branded touch. Pair it with joggers, shorts, or jeans for an effortlessly cool everyday look."
image_url,https://images.asos-media.com/products/vans-hotbox-cars-t-shirt-in-brown/205514883-1-chocolate?$n_640w$&wid=513&fit=constrain


# Setup Vantage

Create a new Vantage account (if you don't have one): https://www.vantagediscovery.com/

Vantage console: https://console.vanta.ge/

Vantage platform docs: https://docs.vantagediscovery.com/docs/about

Copy Vantage API key from: https://console.vanta.ge/api

In [11]:
from getpass import getpass

ACCOUNT_ID: str = "YOUR_ACCOUNT_ID"
VANTAGE_API_KEY: str = getpass("Enter your Vantage API key: ")

Enter your Vantage API key:  ········


Create Vantage client:

In [23]:
from vantage_sdk import VantageClient

vantage = VantageClient.using_vantage_api_key(
    vantage_api_key=VANTAGE_API_KEY,
    account_id=ACCOUNT_ID,
)

vantage.get_account()

Account(account_id='novica131', account_name='novica131')

The Vantage platform can use popular models to retrieve item embeddings (or we can calculate them ourselves and upload). In this case, we’ll use OpenAI to get embeddings, so we need to add the OpenAI API key. There are two ways to do this:
1. Use the Vantage Console to add a new Model API Key.
2. Run the cell below.

In [3]:
OPENAI_API_KEY: str = getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key:  ········


In [7]:
model_key = vantage.create_external_key(
    llm_provider = "OpenAI",
    llm_secret = OPENAI_API_KEY,
)

In [14]:
model_key

ExternalKey(external_key_id='81febefa-f1e2-4eb4-bf3a-a941d57010e2', account_id='novica131', external_key_created_date='2024-07-16T21:48:30', llm_provider='OpenAI', llm_secret='sk-YE...gtd', state='Active')

# Create collection
A collection is a fundamental object of the Vantage Platform that enables you to organize, manage, and search your data sets within the platform. Your data records, called documents, are ingested into a collection.

There are 2 types of collections:
1. Vantage Managed Embeddings (VME) - Vantage Platform manage the translation of your data to AI embeddings.
2. User Provided Embeddings (UPE) - You upload embeddings taken from the LLM of your choice into your collection.

We'll create VME collection with OpenAI embeddings:

In [15]:
from vantage_sdk.model.collection import OpenAICollection

collection = OpenAICollection(
    collection_id="test-collection-4",
    embeddings_dimension=1536,
    llm="text-embedding-ada-002",
    external_key_id=model_key.external_key_id
)
created_collection = vantage.create_collection(collection=collection)

/Users/novicasarenac/.pyenv/versions/3.11.4/envs/amsterdam-workshop/lib/python3.11/site-packages/pydantic/main.py:211: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


In [16]:
print(created_collection)

collection_created_time='2024-09-30T06:32:06' collection_status='Pending' collection_state=None collection_id='test-collection-4' user_provided_embeddings=False llm='text-embedding-ada-002' llm_provider='OpenAI' llm_secret=None external_url=None embeddings_dimension=1536 external_key_id='81febefa-f1e2-4eb4-bf3a-a941d57010e2' secondary_external_accounts=None collection_name='Collection [test-collection-4]'


# Indexing

The dataset needs to be processed to match Vantage ingestion format.

The documents we upload into the collection must conform to specific field and format requirements.

<div>
<img src="img/field_format.png" width="700"/>
</div>

Required fields:
1. `id` - This represents your ID for this document. It will be handed back to you in search results.
2. `text` for VME collections - text that will be embedded using your provided model
3. `embeddings` for UPE collections - array of 32-bit floating point numbers. The array length should match the Dimension Size of the collection you're putting data into.

Optional fields:
1. `operation` - `update` (default), `delete`, `add`
2. `meta_` fields - fields prefixed with `meta_` are used for search query filtering
3. `meta_ordered_` fields - fields prefixed with `meta_ordered_` are used for sorting search query results
4. `variants` - used to specify multiple variants of the product

The data can be uploaded:
- As `parquet` or `jsonl` files via Console, SDK, or API
- Using document upload API

### Prepare input data

In [17]:
dataset.head(1)

,id,availability,brand,gender,product_type,style,color,price,price_range,description,image_url,title
0,203506811,InStock,French Connection,Men,Gifts,Seasonal & Niche,BURGUNDY,6.0,Less than 20,Elevate any look with this versatile burgundy ...,https://images.asos-media.com/products/french-...,French Connection polka dot bow tie in burgundy


We'll create `parquet` file and upload it to the platform:

Create filter columns:

In [18]:
vantage_data = dataset.rename(
    columns={
        "availability": "meta_availability",
        "brand": "meta_brand",
        "gender": "meta_gender",
        "product_type": "meta_product_type",
        "style": "meta_style",
        "color": "meta_color",
        "price": "meta_ordered_price",
        "price_range": "meta_price_range",
    }
)
vantage_data.head(1)

,id,meta_availability,meta_brand,meta_gender,meta_product_type,meta_style,meta_color,meta_ordered_price,meta_price_range,description,image_url,title
0,203506811,InStock,French Connection,Men,Gifts,Seasonal & Niche,BURGUNDY,6.0,Less than 20,Elevate any look with this versatile burgundy ...,https://images.asos-media.com/products/french-...,French Connection polka dot bow tie in burgundy


Create text column:

In [19]:
def get_text(row: pd.Series) -> str:
    return "Title: " + row["title"] + "\nDescription: " + row["description"]


vantage_data["text"] = vantage_data.apply(get_text, axis=1)
vantage_data.drop(columns=["description", "image_url", "title"], inplace=True)
vantage_data.head(3)

,id,meta_availability,meta_brand,meta_gender,meta_product_type,meta_style,meta_color,meta_ordered_price,meta_price_range,text
0,203506811,InStock,French Connection,Men,Gifts,Seasonal & Niche,BURGUNDY,6.0,Less than 20,Title: French Connection polka dot bow tie in ...
1,205514883,InStock,Vans,Women,Tops,Casual Wear,CHOCOLATE,18.0,Less than 20,Title: Vans hotbox cars t-shirt in brown\nDesc...
2,204520772,LimitedStock,NaaNaa,Women,Dresses,Dresses & Skirts,BLUE,8.5,Less than 20,Title: NaaNaa satin halter neck mini dress in ...


In [20]:
INPUT_FILE_PATH: str = "data/input.parquet"

vantage_data.to_parquet(INPUT_FILE_PATH, index=False)

### Index data

Upload data into the collection:

In [21]:
vantage.upload_documents_from_parquet_file(
    collection_id=created_collection.collection_id,
    parquet_file_path=INPUT_FILE_PATH
)

200

# Search

Vantage platform provides different types of search:

<div>
<img src="img/search_types.png" width="1000"/>
</div>

1. Semantic search (for VME collections)
2. Embedding search (for UPE collections)
3. More-like-this search
4. More-like-these search
5. Vantage Vibe
6. Shopping assistant

### Semantic search

It takes a text query (typically entered by the end user) and returns IDs and relevance scores of the semantically closest items.

#### **Simple semantic search:**

In [24]:
semantic_results = vantage.semantic_search(
    text="adidas sneakers",
    collection_id=created_collection.collection_id,
)

In [25]:
display_query_results(dataset, semantic_results.results)

#### **Semantic search with filters**

Search results can be filtered by `meta` columns and ordered by `meta_ordered` columns:

In [26]:
from vantage_sdk.model.search import Filter, Sort, Pagination

results_with_filters = vantage.semantic_search(
    text="adidas sneakers",
    collection_id=created_collection.collection_id,
    filter=Filter(boolean_filter="(gender:\"Men\" AND color:\"WHITE\")"),
    sort=Sort(field="price", order="asc", mode="semantic_threshold"),
    pagination=Pagination(page=0, count=10, threshold=10)
)

In [27]:
display_query_results(dataset, results_with_filters.results)

#### **More semantic search examples**

In [28]:
semantic_results_2 = vantage.semantic_search(
    text="outfit for gym cardio sessions",
    filter=Filter(boolean_filter="gender:\"Men\""),
    collection_id=created_collection.collection_id,
)

In [29]:
display_query_results(dataset, semantic_results_2.results)

In [30]:
semantic_results_3 = vantage.semantic_search(
    text="Elegant dinner party outfit",
    collection_id=created_collection.collection_id,
)

In [31]:
display_query_results(dataset, semantic_results_3.results)

In [33]:
semantic_results_4 = vantage.semantic_search(
    text="Light and comfortable clothing for a summer vacation",
    collection_id=created_collection.collection_id,
)

In [34]:
display_query_results(dataset, semantic_results_4.results)

#### More like this

This type of search takes a document ID and finds similar results in your collection.

In [35]:
more_like_this_results = vantage.more_like_this_search(
    document_id="205357122",
    collection_id="test-collection-3"
)

In [36]:
display_query_results(dataset, more_like_this_results.results)

#### More like these

<div>
<img src="img/more_like_these.png" width="800"/>
</div>

This type of search blends `text`, `items`, and `embeddings` to get personalized search, usually based on user data or external sources. 
It allows combining multi-modal inputs, such as blending image embeddings or text descriptions with current searches.

In [37]:
from vantage_sdk.model.search import MoreLikeTheseItem

these = [
    MoreLikeTheseItem(query_text="outfit for long walks", weight=0.4),
    MoreLikeTheseItem(query_document_id="205629611", weight=0.4),
    MoreLikeTheseItem(query_document_id="206263911", weight=0.2)
]
more_like_these_results = vantage.more_like_these_search(
    more_like_these=these,
    collection_id=created_collection.collection_id,
    filter=Filter(boolean_filter="gender:\"Men\""),
)

In [38]:
display_query_results(dataset, more_like_these_results.results)

#### **Vantage Vibe**

This feature allows you to search over collections with a specific "vibe" or thematic focus. This feature leverages both visual and semantic inputs to refine and personalize search results.

We need to create a Vibe configuration first:

In [39]:
vibe = vantage.create_vibe_configuration(
    name="Floral vibe",
    external_account_id=model_key.external_key_id,
    llm_model_name="gpt-4o-mini",
)
vibe

ForbiddenException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'vary': 'Origin,Access-Control-Request-Method,Access-Control-Request-Headers', 'x-content-type-options': 'nosniff', 'x-xss-protection': '0', 'cache-control': 'no-cache, no-store, max-age=0, must-revalidate', 'pragma': 'no-cache', 'expires': '0', 'strict-transport-security': 'max-age=31536000 ; includeSubDomains', 'x-frame-options': 'DENY', 'content-type': 'application/problem+json', 'date': 'Mon, 30 Sep 2024 06:57:20 GMT', 'transfer-encoding': 'chunked'})
HTTP response body: {"type":"about:blank","title":"Insufficient Permissions","status":403,"detail":"Sorry, account novica131 is not an enterprise account","instance":"/v1/account/novica131/vibe","properties":{"timestamp":"2024-09-30T06:57:20.417917200Z"}}


Images that will be used for Vibe search:

In [40]:
VIBE_IMAGES: List[str] = [
    "https://cdna.lystit.com/520/650/n/photos/asos/bc43b994/nobodys-child-Multi-floral-Aurora-Bloom-Bandeau-Midi-Dress.jpeg",
    "https://product-images.thecoolhour.com/images/asos_annorlunda_annorlunda_barb_wire_floral_asymmetric_fluted_sleeve_midaxi_dress_in_pink_xl.jpg",
    "https://images.asos-media.com/products/french-connection-long-sleeve-mini-mesh-dress-in-white-floral/204319668-1-whitefloral?$n_960w$&wid=952&fit=constrain",
    "https://is4.revolveassets.com/images/p4/n/z/ASTR-WD159_V1.jpg"
]

display_images(VIBE_IMAGES)

Vibe search:

In [178]:
from vantage_sdk.model.search import VantageVibeImageUrl

images = [VantageVibeImageUrl(url=image) for image in VIBE_IMAGES]
text = "romantic dinner"

vibe_results = vantage.vantage_vibe_search(
    vibe_id=vibe.id,
    collection_id=created_collection.collection_id,
    images=images,
    text=text,
)

In [179]:
display_query_results(dataset, vibe_results.results)

#### **Shopping assistant**

With shopping assistant, users can create a detailed specification that defines how search results should be grouped. These groups are created based on the criteria set during the assistant's creation. Once the assistant is set up, users can perform searches by sending a `text query` and a `shopping_assistant_id`.

We need to create a Shopping assistant configuration first:

In [182]:
shopping_assistant = vantage.create_shopping_assistant(
    name="Full outfit assistant",
    groups=["shirt", "pants", "shoes"],
    external_account_id=model_key.external_key_id,
    llm_model_name="gpt-4o-mini",
)

In [183]:
shopping_assistant

ShoppingAssistant(shopping_assistant_id='5c1badf1-629d-4eb5-abf5-a0236a8dfac8', account_id='novica131', name='Full outfit assistant', groups=['shirt', 'pants', 'shoes'], external_account_id='c0c4514e-69ee-4cd9-ab8b-ef08155dd33c', llm_model_name='gpt-4o-mini', system_prompt_id=None)

Query using shopping assistant:

In [197]:
response = vantage.shopping_assistant_search(
    collection_id=created_collection.collection_id,
    text="outfit for business meeting",
    shopping_assistant_id=shopping_assistant.shopping_assistant_id,
    max_groups=3,
    filter=Filter(boolean_filter="gender:\"Men\""),
)

In [187]:
response

ShoppingAssistantResult(request_id=1727036425163, status=200, message='OK', results=None, facets=None, groups=[ShoppingAssistantGroupResult(group_id='shirt', group_name='Professional Shirts', results=[ShoppingAssistantGroupResultResultsInner(id='202684240', score=0.9231213331222534, sort_score=0.9231213331222534), ShoppingAssistantGroupResultResultsInner(id='204266858', score=0.9174026846885681, sort_score=0.9174026846885681), ShoppingAssistantGroupResultResultsInner(id='10039576', score=0.9154284000396729, sort_score=0.9154284000396729), ShoppingAssistantGroupResultResultsInner(id='204944601', score=0.914712131023407, sort_score=0.914712131023407), ShoppingAssistantGroupResultResultsInner(id='206056577', score=0.9146819710731506, sort_score=0.9146819710731506), ShoppingAssistantGroupResultResultsInner(id='203455725', score=0.9146536588668823, sort_score=0.9146536588668823), ShoppingAssistantGroupResultResultsInner(id='202969656', score=0.9145739674568176, sort_score=0.9145739674568176

In [222]:
display_shopping_assistant_results(dataset, response.groups)